<a href="https://colab.research.google.com/github/UznetDev/Kabr-prediction/blob/main/test_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, KFold
from sklearn.linear_model import *
from sklearn.preprocessing import *
from sklearn.feature_selection import *
from sklearn.metrics import *
from sklearn.ensemble import *
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
import warnings
# warnings.filterwarnings('ignore')

Feature ingenering

In [2]:
def feature_funk(df):
    df['Surface Area'] = 2 * (df['Length'] * df['Diameter'] + df['Length'] * df['Height'] + df['Diameter'] * df['Height'])
    df['Vol'] = df['Length'] * df['Diameter'] * df['Height']
    df['Vol_Hgt'] = (4/3) * np.pi * (df['Length']/2) * (df['Diameter']/2) * (df['Height']/2)

    df['Log_Wt'] = df['Weight'].agg(lambda x: np.log(x + 1) if x > 0 else np.nan)
    df['Length Bins'] = pd.qcut(df['Length'], q=4, labels=False)

    df['Wt_Red_Aft_Shk'] = df['Weight'] - df['Shucked Weight']
    df['Vis_Wt_Prop'] = df['Viscera Weight'] / df['Weight']

    df['Len_to_Dia_Ratio'] = df['Length'] / df['Diameter']
    df['Hgt_to_Wt_Ratio'] = df['Height'] / df['Weight']

    sex = {'I': 1, 'F': 3, 'M': 2}
    df['Sex'] = df['Sex'].replace(sex)
    df = df[df['Sex'].isin([1, 2, 3])]
    df.loc[:, 'Sex'] = pd.to_numeric(df['Sex'], errors='coerce')

    return df

train_df = feature_funk(pd.read_csv('train.csv'))

<ipython-input-2-d579729c8722>:6: FutureWarning: using <function feature_funk.<locals>.<lambda> at 0x7e37230d79a0> in Series.agg cannot aggregate and has been deprecated. Use Series.transform to keep behavior unchanged.
  df['Log_Wt'] = df['Weight'].agg(lambda x: np.log(x + 1) if x > 0 else np.nan)


Training model

In [3]:
X = train_df.drop(columns=['id', 'Age'])
y = train_df['Age']

preprocessor = ColumnTransformer(
    transformers=[
        ('num', Pipeline([
            ('imputer', SimpleImputer(strategy='mean')),
            ('poly', PolynomialFeatures(degree=2, include_bias=False, interaction_only=True)),
            ('scaler', StandardScaler())
        ]), X.columns),
    ])
cv = KFold(n_splits=5, shuffle=True, random_state=42)



pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', HuberRegressor(alpha=0.0001, max_iter=100000))

])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

cv_scores = cross_val_score(pipeline, X_train, y_train, cv=cv, scoring='neg_mean_absolute_error')


mean_cv_mae = -np.mean(cv_scores)
pipeline.fit(X_train, y_train)

y_pred = pipeline.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)

print(f'CV RMAE: {np.mean(-cv_scores):.4f}')
print(f'Test MAE: {mae:.4f}')
print(f'Test MSE: {mse:.4f}')
print(f'Test R^2 Score: {r2:.4f}')

CV RMAE: 1.3320
Test MAE: 1.3118
Test MSE: 3.8770
Test R^2 Score: 0.6136
